In [ ]:
dff = df[df['stroke'] == 1]  
dff

In [ ]:
dff = df[df['stroke'] == 1]  
dff

In [ ]:
dfstr = df[df['heart_disease'] == 1]
dfstr

In [ ]:
dfstrss = dfstr[dfstr['stroke'] == 1]
dfstrss


In [ ]:
len(dfstrss)
